In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import lightgbm as lg

In [3]:
# Reading the Dataset
dataset = pd.read_csv("Preprocessed_project_risk.csv")
dataset

,Team_Size,Project_Budget_USD,Estimated_Timeline_Months,Complexity_Score,Stakeholder_Count,Past_Similar_Projects,External_Dependencies_Count,Change_Request_Frequency,Team_Turnover_Rate,Vendor_Reliability_Score,...,Tech_Environment_Stability,Contract_Type,Resource_Contention_Level,Industry_Volatility,Client_Experience_Level,Change_Control_Maturity,Risk_Management_Maturity,Team_Colocation,Documentation_Quality,Risk_Level
0,32.0,1526276.550,32.0,9.70,16.0,3.0,3.0,1.05,0.16,0.84,...,0,Time & Materials,High,Extreme,First-time,Basic,Basic,Fully Colocated,Good,High
1,2.0,390790.150,9.0,2.72,9.0,0.0,2.0,2.61,0.42,0.79,...,0,Cost-Plus,Low,Stable,Occasional,Advanced,Formal,Fully Remote,Poor,Low
2,2.0,246674.760,6.0,2.04,7.0,1.0,0.0,0.83,0.55,0.89,...,0,Cost-Plus,High,Stable,Regular,0,0,Hybrid,Good,Medium
3,12.0,1427830.630,17.0,7.54,16.0,0.0,5.0,2.42,0.33,0.84,...,Mixed,Hybrid,High,Extreme,Strategic,Formal,Basic,Hybrid,Basic,High
4,24.0,1696746.640,24.0,6.68,17.0,0.0,2.0,0.16,0.36,0.86,...,0,Cost-Plus,High,Moderate,Occasional,Basic,0,Partially Colocated,Basic,High
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3995,9.0,731548.970,14.0,5.40,9.0,4.0,3.0,1.09,0.25,0.94,...,Legacy/Unstable,Fixed-Price,Medium,Moderate,Occasional,0,0,Hybrid,Basic,High
3996,9.0,492981.400,9.0,2.69,8.0,5.0,3.0,2.11,0.06,0.71,...,0,Fixed-Price,Medium,Stable,Occasional,0,Basic,Hybrid,Basic,Medium
3997,12.0,1097040.650,24.0,8.82,10.0,2.0,4.0,0.52,0.23,0.95,...,Legacy/Unstable,Time & Materials,Low,Moderate,Regular,0,Basic,Fully Remote,Basic,High
3998,26.0,2650876.115,28.0,10.00,23.0,1.0,6.0,1.19,0.30,0.50,...,0,Fixed-Price,Medium,High,Strategic,Basic,Formal,Fully Remote,Good,High


In [4]:
# Separate target first
dependent = dataset["Risk_Level"]
independent = dataset.drop(columns=["Risk_Level"])

# Apply get_dummies only on independent
X_encoded = pd.get_dummies(independent, drop_first=True)
X_encoded=X_encoded.replace({True:1,False:0})
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
Y_encoded = le.fit_transform(dependent)
print(X_encoded.shape)
print(Y_encoded.shape)

C:\Users\PRIYANKA MANI\AppData\Local\Temp\ipykernel_11484\4170489483.py:7: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  X_encoded=X_encoded.replace({True:1,False:0})


(4000, 4098)
(4000,)


In [18]:
from sklearn.feature_selection import SelectKBest, f_classif

selector = SelectKBest(score_func=f_classif, k=10)
X_anova = selector.fit_transform(X_encoded, Y_encoded)

selected_features = X_encoded.columns[selector.get_support()]
X_selected = X_encoded[selected_features]
X_selected

C:\Anaconda3\Lib\site-packages\sklearn\feature_selection\_univariate_selection.py:112: UserWarning: Features [24] are constant.
  warnings.warn("Features %s are constant." % constant_features_idx, UserWarning)
C:\Anaconda3\Lib\site-packages\sklearn\feature_selection\_univariate_selection.py:113: RuntimeWarning: invalid value encountered in divide
  f = msb / msw


,Estimated_Timeline_Months,Complexity_Score,External_Dependencies_Count,Team_Turnover_Rate,Project_Type_R&D,Technology_Familiarity_New,Org_Process_Maturity_Optimizing,Resource_Contention_Level_Low,Change_Control_Maturity_Advanced,Risk_Management_Maturity_Advanced
0,32.0,9.70,3.0,0.16,0,0,0,0,0,0
1,9.0,2.72,2.0,0.42,0,0,1,1,1,0
2,6.0,2.04,0.0,0.55,0,0,0,0,0,0
3,17.0,7.54,5.0,0.33,0,0,0,0,0,0
4,24.0,6.68,2.0,0.36,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...
3995,14.0,5.40,3.0,0.25,0,0,0,0,0,0
3996,9.0,2.69,3.0,0.06,0,0,0,0,0,0
3997,24.0,8.82,4.0,0.23,0,1,0,1,0,0
3998,28.0,10.00,6.0,0.30,1,1,0,0,0,0


In [19]:
dependent

0         High
1          Low
2       Medium
3         High
4         High
         ...  
3995      High
3996    Medium
3997      High
3998      High
3999    Medium
Name: Risk_Level, Length: 4000, dtype: object

In [20]:
corr_matrix=X_selected.corr()
corr_matrix

,Estimated_Timeline_Months,Complexity_Score,External_Dependencies_Count,Team_Turnover_Rate,Project_Type_R&D,Technology_Familiarity_New,Org_Process_Maturity_Optimizing,Resource_Contention_Level_Low,Change_Control_Maturity_Advanced,Risk_Management_Maturity_Advanced
Estimated_Timeline_Months,1.000000,0.792578,0.553153,0.034673,0.348950,0.095899,-0.027171,-0.197752,-0.029861,-0.019626
Complexity_Score,0.792578,1.000000,0.704350,0.028492,0.507182,0.144947,0.010950,-0.083335,-0.031757,0.003401
External_Dependencies_Count,0.553153,0.704350,1.000000,0.038844,0.366769,0.105751,0.011110,-0.065264,-0.031421,0.012534
Team_Turnover_Rate,0.034673,0.028492,0.038844,1.000000,0.028182,0.088787,-0.012480,-0.003737,-0.016531,-0.030922
Project_Type_R&D,0.348950,0.507182,0.366769,0.028182,1.000000,0.302831,-0.040644,-0.058947,-0.057600,-0.028488
Technology_Familiarity_New,0.095899,0.144947,0.105751,0.088787,0.302831,1.000000,-0.022505,-0.027873,-0.033380,-0.015030
Org_Process_Maturity_Optimizing,-0.027171,0.010950,0.011110,-0.012480,-0.040644,-0.022505,1.000000,0.048406,0.361300,0.402008
Resource_Contention_Level_Low,-0.197752,-0.083335,-0.065264,-0.003737,-0.058947,-0.027873,0.048406,1.000000,0.033268,0.039757
Change_Control_Maturity_Advanced,-0.029861,-0.031757,-0.031421,-0.016531,-0.057600,-0.033380,0.361300,0.033268,1.000000,0.164575
Risk_Management_Maturity_Advanced,-0.019626,0.003401,0.012534,-0.030922,-0.028488,-0.015030,0.402008,0.039757,0.164575,1.000000


In [21]:
#split into training set and test
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X_selected, Y_encoded, test_size = 1/3, random_state = 0)
X_train, X_test, Y_train, Y_test

(      Estimated_Timeline_Months  Complexity_Score  \
 1196                       18.0              5.10   
 1931                       13.0              4.91   
 121                         6.0              2.96   
 3007                        6.0              3.38   
 3857                       22.0              5.90   
 ...                         ...               ...   
 835                        26.0             10.00   
 3264                        6.0              3.13   
 1653                       11.0              5.02   
 2607                        9.0              4.90   
 2732                       20.0              7.98   
 
       External_Dependencies_Count  Team_Turnover_Rate  Project_Type_R&D  \
 1196                          1.0                0.23                 0   
 1931                          1.0                0.07                 0   
 121                           3.0                0.27                 0   
 3007                          0.0            

In [22]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [23]:
from lightgbm  import LGBMClassifier
from sklearn.model_selection import GridSearchCV

param_grid = {'criterion':['gini','entropy'],
              'max_features': ['auto','sqrt','log2'],
              'n_estimators':[10,100]} 
grid = GridSearchCV(LGBMClassifier(), param_grid, refit = True, verbose = 3,n_jobs=-1,scoring='f1_weighted') 
   
# fitting the model for grid search 
grid.fit(X_train, Y_train) 

Fitting 5 folds for each of 12 candidates, totalling 60 fits
[LightGBM] [Warning] Unknown parameter: criterion
[LightGBM] [Warning] Unknown parameter: max_features
[LightGBM] [Warning] Unknown parameter: criterion
[LightGBM] [Warning] Unknown parameter: max_features
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001072 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 393
[LightGBM] [Info] Number of data points in the train set: 2666, number of used features: 10
[LightGBM] [Info] Start training from score -1.673726
[LightGBM] [Info] Start training from score -1.338684
[LightGBM] [Info] Start training from score -1.576600
[LightGBM] [Info] Start training from score -1.068318


GridSearchCV(estimator=LGBMClassifier(), n_jobs=-1,
             param_grid={'criterion': ['gini', 'entropy'],
                         'max_features': ['auto', 'sqrt', 'log2'],
                         'n_estimators': [10, 100]},
             scoring='f1_weighted', verbose=3)

In [24]:
# print best parameter after tuning 
print(grid.best_params_) 
result=grid.cv_results_
print(result)
grid_predictions = grid.predict(X_test) 
   

from sklearn.metrics import confusion_matrix
cm = confusion_matrix(Y_test, grid_predictions)



# print classification report 
from sklearn.metrics import classification_report
clf_report = classification_report(Y_test, grid_predictions)

{'criterion': 'gini', 'max_features': 'auto', 'n_estimators': 10}
{'mean_fit_time': array([ 0.89512329,  8.01728029,  0.85281549,  8.08572974,  0.86448512,
       10.17607455,  0.89028802, 11.84512696,  0.89220753, 12.3116291 ,
        1.55804605,  8.74063067]), 'std_fit_time': array([0.04212613, 0.07093509, 0.00905402, 0.04658151, 0.04274062,
       1.71244514, 0.01530458, 0.94409473, 0.07539714, 1.23944654,
       0.15120075, 0.49253677]), 'mean_score_time': array([0.00546393, 0.01045861, 0.00654402, 0.00881715, 0.00379014,
       0.01207442, 0.0058567 , 0.01278439, 0.00846987, 0.0104815 ,
       0.00394387, 0.01269965]), 'std_score_time': array([0.00484869, 0.00241887, 0.00367665, 0.0013249 , 0.00416309,
       0.00443635, 0.0017041 , 0.0044988 , 0.00426244, 0.00274448,
       0.0046983 , 0.00121263]), 'param_criterion': masked_array(data=['gini', 'gini', 'gini', 'gini', 'gini', 'gini',
                   'entropy', 'entropy', 'entropy', 'entropy', 'entropy',
                   'ent

In [25]:
from sklearn.metrics import f1_score
f1_macro=f1_score(Y_test,grid_predictions,average='weighted')
print("The f1_macro value for best parameter {}:".format(grid.best_params_),f1_macro)

The f1_macro value for best parameter {'criterion': 'gini', 'max_features': 'auto', 'n_estimators': 10}: 0.43704873624249224


In [26]:
print("The confusion Matrix:\n",cm)

The confusion Matrix:
 [[117  58   4  83]
 [ 59  90  12 176]
 [  2  19  82 152]
 [ 28  73  69 310]]


In [27]:
print("The report:\n",clf_report)

The report:
               precision    recall  f1-score   support

           0       0.57      0.45      0.50       262
           1       0.38      0.27      0.31       337
           2       0.49      0.32      0.39       255
           3       0.43      0.65      0.52       480

    accuracy                           0.45      1334
   macro avg       0.47      0.42      0.43      1334
weighted avg       0.45      0.45      0.44      1334



In [28]:
from sklearn.metrics import roc_auc_score

y_prob = grid.predict_proba(X_test)

roc_auc = roc_auc_score(
    Y_test,
    y_prob,
    multi_class="ovr",   # or "ovo"
    average="weighted"
)

print("ROC AUC:", roc_auc)

[LightGBM] [Warning] Unknown parameter: criterion
[LightGBM] [Warning] Unknown parameter: max_features
ROC AUC: 0.686963812832579


In [29]:
X_selected.columns

Index(['Estimated_Timeline_Months', 'Complexity_Score',
       'External_Dependencies_Count', 'Team_Turnover_Rate', 'Project_Type_R&D',
       'Technology_Familiarity_New', 'Org_Process_Maturity_Optimizing',
       'Resource_Contention_Level_Low', 'Change_Control_Maturity_Advanced',
       'Risk_Management_Maturity_Advanced'],
      dtype='object')

In [17]:
best_model = grid.best_estimator_
importance_of_each_feature=best_model.feature_importances_
importance_of_each_feature

array([192,  89, 225,  66, 262,   6,  85,  52,  30,  67,  28,  45,  53])